📌 Датасети

Працюємо з файлами з бакету:

s3://<your-bucket-name>/nyc-taxi/

📌 Завдання

✅ 1. External Stage

* Створіть External Stage, який підключено до публічного бакету
* Перевірте доступність файлів за допомогою LIST @stage_name

✅ 2. Завантаження сирих даних

* Створіть таблиці yellow_raw, green_raw, taxi_zone_lookup
* Завантажте файли в таблиці за допомогою команди COPY INTO

✅ 3. Збагачення даних

* Створіть таблиці yellow_enriched та green_enriched
* Реалізуйте JOIN з lookup таблицею
* Pickup/Dropoff зони (zone_name)
* Використовуйте LEFT JOIN для збереження всіх записів

✅ 4. Трансформації

* Фільтруйте записи:
  * trip_distance > 0
  * total_amount > 0
  * passenger_count між 1 та 6
* Додайте колонку trip_category:
  * Short (до 2 км), Medium (2–10 км), Long (>10 км)
* Додайте колонку pickup_hour: годину з pickup_datetime
* Створіть агреговану таблицю з підрахунком по зонах

✅ 5. Time Travel

* Видаліть кілька записів зі збагаченої таблиці (наприклад, green_enriched)
* За допомогою Time Travel:
  * Перевірте стару версію таблиці (через AT або BEFORE)
  * Відновіть видалені записи у нову таблицю або у ту ж (через INSERT SELECT)

✅ 6. Streams

* Створіть Stream на таблиці yellow_enriched
* Додайте нові записи вручну або через COPY INTO
* Перевірте, що Stream відображає INSERT/UPDATE зміни
* Створіть цільову таблицю yellow_changes_log для зберігання змін

✅ 7. Tasks (batch/streaming jobs)

* Створіть Task, який щогодини:
  * Зчитує зміни зі Stream
  * Вставляє нові записи у yellow_changes_log
* Використайте SCHEDULE = '1 HOUR' або WAREHOUSE = '...' для запуску
* Зробіть окремий Task для агрегованої статистики:
  * Середня відстань
  * Середня ціна
  * Кількість поїздок
  * Зберігайте у таблицю zone_hourly_stats
* Використайте SCHEDULE = '1 HOUR' або WAREHOUSE = '...' для запуску

📍 8. Додаткове завдання (опціонально)

* Об’єднайте yellow_enriched та green_enriched у таблицю all_trips
* Створіть Stored Procedure:
  * Перевірка на дублікати перед вставкою
  * Запис результату в лог-таблицю
* Налаштуйте Zero-Copy Cloning бази для створення середовища taxi_dev

In [ ]:
CREATE OR REPLACE STORAGE INTEGRATION lavreniuk_s3_integration
  TYPE = EXTERNAL_STAGE
  STORAGE_PROVIDER = 'S3'
  ENABLED = TRUE
  STORAGE_AWS_ROLE_ARN = 'arn:aws:iam::554739427960:role/lavreniuk-snowflake'
  STORAGE_ALLOWED_LOCATIONS = ('*');

In [ ]:
DESC INTEGRATION s3_integration;

In [ ]:
-- creating format

CREATE OR REPLACE FILE FORMAT csv_format
    TYPE = 'CSV'
    COMPRESSION = 'AUTO'
    FIELD_DELIMITER = ','
    RECORD_DELIMITER = '\n'
    SKIP_HEADER = 1
    TRIM_SPACE = TRUE,
    FIELD_OPTIONALLY_ENCLOSED_BY='"'
    ERROR_ON_COLUMN_COUNT_MISMATCH = FALSE;

CREATE OR REPLACE FILE FORMAT parquet_format
    TYPE = 'PARQUET'
    COMPRESSION = AUTO
    USE_LOGICAL_TYPE = TRUE
    TRIM_SPACE = TRUE;


In [ ]:
 CREATE OR REPLACE STAGE lavreniuk_s3_stage
    STORAGE_INTEGRATION = lavreniuk_s3_integration
    URL = 's3://lavreniuk-hw3/nyc_taxi/'
    FILE_FORMAT = csv_format;

In [ ]:
LIST @lavreniuk_s3_stage;

In [ ]:
-- Creating raw tables

CREATE TABLE IF NOT EXISTS yellow_raw(
    VendorID BIGINT,
    passenger_count BIGINT,
    trip_distance DOUBLE,
    RatecodeID BIGINT,
    store_and_fwd_flag VARCHAR,
    PULocationID BIGINT,
    DOLocationID BIGINT,
    payment_type BIGINT,
    fare_amount DOUBLE,
    extra DOUBLE,
    mta_tax DOUBLE,
    tip_amount DOUBLE,
    tolls_amount DOUBLE,
    improvement_surcharge DOUBLE,
    total_amount DOUBLE,
    congestion_surcharge DOUBLE,
    tpep_pickup_datetime TIMESTAMP_NTZ,
    tpep_dropoff_datetime TIMESTAMP_NTZ,
    Airport_fee DOUBLE
);

CREATE TABLE IF NOT EXISTS green_raw(
    VendorID BIGINT,
    passenger_count BIGINT,
    trip_distance DOUBLE,
    RatecodeID BIGINT,
    store_and_fwd_flag VARCHAR,
    PULocationID BIGINT,
    DOLocationID BIGINT,
    payment_type BIGINT,
    fare_amount DOUBLE,
    extra DOUBLE,
    mta_tax DOUBLE,
    tip_amount DOUBLE,
    tolls_amount DOUBLE,
    improvement_surcharge DOUBLE,
    total_amount DOUBLE,
    congestion_surcharge DOUBLE,
    lpep_pickup_datetime TIMESTAMP_NTZ,
    lpep_dropoff_datetime TIMESTAMP_NTZ
);

CREATE TABLE IF NOT EXISTS taxi_zone_lookup(
    LocationID BIGINT,
    Borough VARCHAR,
    Zone VARCHAR,
    service_zone VARCHAR
);

In [ ]:
-- seeding yellow

COPY INTO public.yellow_raw
    FROM @lavreniuk_s3_stage/yellow
    FILE_FORMAT = parquet_format
    MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
    ON_ERROR = 'CONTINUE';

In [ ]:
-- seeding green

COPY INTO public.green_raw
    FROM @lavreniuk_s3_stage/green
    FILE_FORMAT = parquet_format
    MATCH_BY_COLUMN_NAME = CASE_INSENSITIVE
    ON_ERROR = 'CONTINUE';

In [ ]:
-- seeding taxi_zone_lookup

COPY INTO public.taxi_zone_lookup
    FROM @lavreniuk_s3_stage/taxi_zone_lookup.csv
    FILE_FORMAT = csv_format
    ON_ERROR = 'CONTINUE';

In [ ]:
SELECT * FROM green_raw LIMIT 5;

In [ ]:
-- Створіть таблиці yellow_enriched та green_enriched
-- * Реалізуйте JOIN з lookup таблицею
-- * Pickup/Dropoff зони (zone_name)
-- * Використовуйте LEFT JOIN для збереження всіх записів

CREATE TABLE yellow_enriched AS 
SELECT
    yr.*,
    zl_pi.Zone as pickup_zone,
    zl_do.Zone as dropoff_zone
FROM yellow_raw yr
LEFT JOIN taxi_zone_lookup zl_pi
    ON yr.PULocationID = zl_pi.LocationID
LEFT JOIN taxi_zone_lookup zl_do
    ON yr.DOLocationID = zl_do.LocationID

In [ ]:
CREATE TABLE green_enriched AS 
SELECT
    gr.*,
    zl_pi.Zone as pickup_zone,
    zl_do.Zone as dropoff_zone
FROM green_raw gr
LEFT JOIN taxi_zone_lookup zl_pi
    ON gr.PULocationID = zl_pi.LocationID
LEFT JOIN taxi_zone_lookup zl_do
    ON gr.DOLocationID = zl_do.LocationID

In [ ]:
SELECT TOP 5 * FROM green_enriched

In [ ]:
SELECT TOP 5 * FROM yellow_enriched

In [ ]:
-- * Фільтруйте записи:
--   * trip_distance > 0
--   * total_amount > 0
--   * passenger_count між 1 та 6
-- * Додайте колонку trip_category:
--   * Short (до 2 км), Medium (2–10 км), Long (>10 км)
-- * Додайте колонку pickup_hour: годину з pickup_datetime
-- * Створіть агреговану таблицю з підрахунком по зонах

CREATE TABLE green_enriched_preprocessed AS
SELECT
    *,
    CASE 
        WHEN trip_distance < 2 
            THEN 'Short'
        WHEN trip_distance >= 2 AND trip_distance <= 10
            THEN 'Medium'
        ELSE
            'Long'
    END as trip_category,
    EXTRACT(HOUR from lpep_pickup_datetime) as pickup_hour
FROM green_enriched
WHERE
    trip_distance > 0
    AND total_amount > 0
    AND passenger_count BETWEEN 1 AND 6;

CREATE TABLE yellow_enriched_preprocessed AS
SELECT
    *,
    CASE 
        WHEN trip_distance < 2 
            THEN 'Short'
        WHEN trip_distance >= 2 AND trip_distance <= 10
            THEN 'Medium'
        ELSE
            'Long'
    END as trip_category,
    EXTRACT(HOUR from tpep_pickup_datetime) as pickup_hour
FROM yellow_enriched
WHERE
    trip_distance > 0
    AND total_amount > 0
    AND passenger_count BETWEEN 1 AND 6;

In [ ]:
SELECT TOP 5 * FROM green_enriched_preprocessed

In [ ]:
CREATE TABLE green_zone_aggregated AS
SELECT
    pickup_zone,
    AVG(trip_distance) as avg_trip_distance,
    AVG(total_amount) as avg_total_amount,
    AVG(tip_amount) AS avg_tip_amount,
    MAX(trip_distance) AS max_trip_distance,
    MAX(tip_amount) AS max_tip_amount
FROM green_enriched_preprocessed
GROUP BY pickup_zone;

CREATE TABLE yellow_zone_aggregated AS
SELECT
    pickup_zone,
    AVG(trip_distance) as avg_trip_distance,
    AVG(total_amount) as avg_total_amount,
    AVG(tip_amount) AS avg_tip_amount,
    MAX(trip_distance) AS max_trip_distance,
    MAX(tip_amount) AS max_tip_amount
FROM yellow_enriched_preprocessed
GROUP BY pickup_zone;

In [ ]:
SELECT TOP 5 * FROM green_zone_aggregated

In [ ]:
-- * Видаліть кілька записів зі збагаченої таблиці (наприклад, green_enriched)
-- * За допомогою Time Travel:
--   * Перевірте стару версію таблиці (через AT або BEFORE)
--   * Відновіть видалені записи у нову таблицю або у ту ж (через INSERT SELECT)

SELECT COUNT(1) FROM green_enriched_preprocessed WHERE PULocationID = 226 AND DOLocationID = 82 AND pickup_hour = 0 AND trip_category = 'Short' -- random, not very big set for removing

In [ ]:
DELETE FROM green_enriched_preprocessed WHERE PULocationID = 226 AND DOLocationID = 82 AND pickup_hour = 0 AND trip_category = 'Short'

In [ ]:
SELECT COUNT(1) FROM green_enriched_preprocessed AT(OFFSET => -60*30);

In [ ]:
SELECT COUNT(1) FROM green_enriched_preprocessed;

In [ ]:
-- Different count, so lets restore deleted data
CREATE OR REPLACE TABLE green_enriched_preprocessed AS
    SELECT * FROM green_enriched_preprocessed AT(OFFSET => -60*30);

In [ ]:
SELECT COUNT(1) FROM green_enriched_preprocessed;

In [ ]:
-- * Створіть Stream на таблиці yellow_enriched
-- * Додайте нові записи вручну або через COPY INTO
-- * Перевірте, що Stream відображає INSERT/UPDATE зміни
-- * Створіть цільову таблицю yellow_changes_log для зберігання змін

CREATE OR REPLACE STREAM yellow_changes_log 
    ON TABLE yellow_enriched;

In [ ]:
DESCRIBE TABLE yellow_enriched;

In [ ]:
INSERT INTO yellow_enriched(VendorID) VALUES (11111), (22222);

In [ ]:
UPDATE yellow_enriched SET VendorID = 55555 WHERE VendorID = 11111;

In [ ]:
SELECT * FROM yellow_changes_log; -- read it few times, to ensure the stream become empty

In [ ]:
 -- (up) insert and update executed 2 times 
 CREATE TABLE yellow_accumulate_log_table AS
    SELECT * FROM yellow_changes_log; -- Створіть цільову таблицю для зберігання змін (назвав по іншому, бо зробив цей неймінг для стріму)


In [ ]:
SELECT * FROM yellow_accumulate_log_table;

In [ ]:
-- * Створіть Task, який щогодини:
--   * Зчитує зміни зі Stream
--   * Вставляє нові записи у yellow_changes_log
-- * Використайте SCHEDULE = '1 HOUR' або WAREHOUSE = '...' для запуску
-- * Зробіть окремий Task для агрегованої статистики:
--   * Середня відстань
--   * Середня ціна
--   * Кількість поїздок
--   * Зберігайте у таблицю zone_hourly_stats
-- * Використайте SCHEDULE = '1 HOUR' або WAREHOUSE = '...' для запуску

In [ ]:
CREATE TASK history_stream_filling_task 
    SCHEDULE = '1 HOUR' AS
        INSERT INTO yellow_accumulate_log_table
            SELECT * FROM yellow_changes_log;

In [ ]:
CREATE TABLE zone_hourly_stats AS
SELECT TOP 1
    GETDATE() as ts,
    pickup_hour,
    COUNT(1) AS total_trips,
    AVG(trip_distance) AS avg_tip_distance,
    AVG(total_amount) AS avg_total_amount
FROM yellow_enriched_preprocessed
WHERE pickup_hour IS NOT NULL
GROUP BY pickup_hour


In [ ]:
TRUNCATE TABLE zone_hourly_stats; -- clear before task creation
DESCRIBE TABLE zone_hourly_stats;

In [ ]:
CREATE TASK zone_hourly_stats_task 
    SCHEDULE = '1 HOUR' AS
    INSERT INTO zone_hourly_stats(ts, pickup_hour, total_trips, avg_tip_distance, avg_total_amount)
        SELECT
            GETDATE() as ts,
            pickup_hour,
            COUNT(1) AS total_trips,
            AVG(trip_distance) AS avg_tip_distance,
            AVG(total_amount) AS avg_total_amount
        FROM yellow_enriched_preprocessed
        WHERE pickup_hour IS NOT NULL
        GROUP BY pickup_hour;

In [ ]:
EXECUTE TASK zone_hourly_stats_task; -- check if task work

In [ ]:
SELECT * FROM zone_hourly_stats;

In [ ]:

# * Об’єднайте yellow_enriched та green_enriched у таблицю all_trips
# * Створіть Stored Procedure:
#   * Перевірка на дублікати перед вставкою
#   * Запис результату в лог-таблицю
# * Налаштуйте Zero-Copy Cloning бази для створення середовища taxi_dev
from snowflake.snowpark.context import get_active_session

session = get_active_session()
yellow_enriched_columns = set(session.sql("SELECT TOP 1 * FROM yellow_enriched;").columns)
green_enriched_columns = set(session.sql("SELECT TOP 1 * FROM green_enriched;").columns)
print(f"Yellow: {yellow_enriched_columns}")
print(f"Green: {green_enriched_columns}")
print(yellow_enriched_columns ^ green_enriched_columns)

In [ ]:
CREATE TABLE all_trips AS (
SELECT
    PICKUP_ZONE,
    PASSENGER_COUNT,
    CONGESTION_SURCHARGE,
    TOLLS_AMOUNT,
    TIP_AMOUNT,
    MTA_TAX,
    AIRPORT_FEE,
    TPEP_DROPOFF_DATETIME AS DROPOFF_DATETIME,
    IMPROVEMENT_SURCHARGE,
    DOLOCATIONID,
    TPEP_PICKUP_DATETIME AS PICKUP_DATETIME, 
    VENDORID,
    STORE_AND_FWD_FLAG,
    EXTRA,
    FARE_AMOUNT,
    TOTAL_AMOUNT,
    PAYMENT_TYPE,
    TRIP_DISTANCE,
    DROPOFF_ZONE,
    RATECODEID,
    PULOCATIONID,
    'YELLOW' AS taxi_color
FROM yellow_enriched
    
UNION ALL

SELECT 
    PICKUP_ZONE,
    PASSENGER_COUNT,
    CONGESTION_SURCHARGE,
    TOLLS_AMOUNT,
    TIP_AMOUNT,
    MTA_TAX,
    NULL AS AIRPORT_FEE,
    LPEP_DROPOFF_DATETIME AS DROPOFF_DATETIME,
    IMPROVEMENT_SURCHARGE,
    DOLOCATIONID,
    LPEP_PICKUP_DATETIME AS PICKUP_DATETIME, 
    VENDORID,
    STORE_AND_FWD_FLAG,
    EXTRA,
    FARE_AMOUNT,
    TOTAL_AMOUNT,
    PAYMENT_TYPE,
    TRIP_DISTANCE,
    DROPOFF_ZONE,
    RATECODEID,
    PULOCATIONID,
    'GREEN' AS taxi_color
FROM
    green_enriched
)

In [ ]:
DESCRIBE TABLE all_trips;

In [ ]:
CREATE OR REPLACE PROCEDURE unduplicated_insert_sp(
      PICKUP_ZONE           VARCHAR,
      PASSENGER_COUNT       NUMBER,
      CONGESTION_SURCHARGE  FLOAT,
      TOLLS_AMOUNT          FLOAT,
      TIP_AMOUNT            FLOAT,
      MTA_TAX               FLOAT,
      AIRPORT_FEE           FLOAT,
      DROPOFF_DATETIME      TIMESTAMP_NTZ,
      IMPROVEMENT_SURCHARGE FLOAT,
      DOLOCATIONID          NUMBER,
      PICKUP_DATETIME       TIMESTAMP_NTZ,
      VENDORID              NUMBER,
      STORE_AND_FWD_FLAG    VARCHAR,
      EXTRA                 FLOAT,
      FARE_AMOUNT           FLOAT,
      TOTAL_AMOUNT          FLOAT,
      PAYMENT_TYPE          NUMBER,
      TRIP_DISTANCE         FLOAT,
      DROPOFF_ZONE          VARCHAR,
      RATECODEID            NUMBER,
      PULOCATIONID          NUMBER,
      TAXI_COLOR            VARCHAR
)
RETURNS STRING
LANGUAGE SQL
AS
$$
DECLARE
        DUPLICATE_EXCEPTION EXCEPTION (-20002, 'HERE IS DUPLICATE!.');
        cnt NUMBER;
BEGIN
        SELECT COUNT(1) INTO :cnt
        FROM all_trips t
        WHERE t.PICKUP_ZONE           = :PICKUP_ZONE
          AND t.PASSENGER_COUNT       = :PASSENGER_COUNT
          AND t.CONGESTION_SURCHARGE  = :CONGESTION_SURCHARGE
          AND t.TOLLS_AMOUNT          = :TOLLS_AMOUNT
          AND t.TIP_AMOUNT            = :TIP_AMOUNT
          AND t.MTA_TAX               = :MTA_TAX
          AND t.AIRPORT_FEE           = :AIRPORT_FEE
          AND t.DROPOFF_DATETIME      = :DROPOFF_DATETIME
          AND t.IMPROVEMENT_SURCHARGE = :IMPROVEMENT_SURCHARGE
          AND t.DOLOCATIONID          = :DOLOCATIONID
          AND t.PICKUP_DATETIME       = :PICKUP_DATETIME
          AND t.VENDORID              = :VENDORID
          AND t.STORE_AND_FWD_FLAG    = :STORE_AND_FWD_FLAG
          AND t.EXTRA                 = :EXTRA
          AND t.FARE_AMOUNT           = :FARE_AMOUNT
          AND t.TOTAL_AMOUNT          = :TOTAL_AMOUNT
          AND t.PAYMENT_TYPE          = :PAYMENT_TYPE
          AND t.TRIP_DISTANCE         = :TRIP_DISTANCE
          AND t.DROPOFF_ZONE          = :DROPOFF_ZONE
          AND t.RATECODEID            = :RATECODEID
          AND t.PULOCATIONID          = :PULOCATIONID
          AND t.TAXI_COLOR            = :TAXI_COLOR;
    IF (cnt = 0)
    THEN
        INSERT INTO all_trips (
            PICKUP_ZONE, PASSENGER_COUNT, CONGESTION_SURCHARGE, TOLLS_AMOUNT,
            TIP_AMOUNT, MTA_TAX, AIRPORT_FEE, DROPOFF_DATETIME, IMPROVEMENT_SURCHARGE,
            DOLOCATIONID, PICKUP_DATETIME, VENDORID, STORE_AND_FWD_FLAG, EXTRA,
            FARE_AMOUNT, TOTAL_AMOUNT, PAYMENT_TYPE, TRIP_DISTANCE, DROPOFF_ZONE,
            RATECODEID, PULOCATIONID, TAXI_COLOR
        )
        VALUES (
            :PICKUP_ZONE, :PASSENGER_COUNT, :CONGESTION_SURCHARGE, :TOLLS_AMOUNT,
            :TIP_AMOUNT, :MTA_TAX, :AIRPORT_FEE, :DROPOFF_DATETIME, :IMPROVEMENT_SURCHARGE,
            :DOLOCATIONID, :PICKUP_DATETIME, :VENDORID, :STORE_AND_FWD_FLAG, :EXTRA,
            :FARE_AMOUNT, :TOTAL_AMOUNT, :PAYMENT_TYPE, :TRIP_DISTANCE, :DROPOFF_ZONE,
            :RATECODEID, :PULOCATIONID, :TAXI_COLOR
        );
    ELSE
        RAISE DUPLICATE_EXCEPTION;
    END IF;

    RETURN 'INSERTED';
END;
$$;

In [ ]:
CALL unduplicated_insert_sp(
    'Midtown',            -- PICKUP_ZONE
    2,                    -- PASSENGER_COUNT
    0.0,                  -- CONGESTION_SURCHARGE
    5.0,                  -- TOLLS_AMOUNT
    3.0,                  -- TIP_AMOUNT
    0.5,                  -- MTA_TAX
    1,                    -- AIRPORT_FEE
    '2025-08-16 14:30:00',-- DROPOFF_DATETIME
    0.3,                  -- IMPROVEMENT_SURCHARGE
    234,                  -- DOLOCATIONID
    '2025-08-16 14:00:00',-- PICKUP_DATETIME
    1,                    -- VENDORID
    'N',                  -- STORE_AND_FWD_FLAG
    0.0,                  -- EXTRA
    25.0,                 -- FARE_AMOUNT
    33.8,                 -- TOTAL_AMOUNT
    1,                    -- PAYMENT_TYPE
    3.5,                  -- TRIP_DISTANCE
    'Downtown',           -- DROPOFF_ZONE
    1,                    -- RATECODEID
    123,                  -- PULOCATIONID
    'GREEN'               -- TAXI_COLOR
);


In [ ]:
CALL unduplicated_insert_sp(
    'Midtown',            -- PICKUP_ZONE
    2,                    -- PASSENGER_COUNT
    0.0,                  -- CONGESTION_SURCHARGE
    5.0,                  -- TOLLS_AMOUNT
    3.0,                  -- TIP_AMOUNT
    0.5,                  -- MTA_TAX
    1,                    -- AIRPORT_FEE
    '2025-08-16 14:30:00',-- DROPOFF_DATETIME
    0.3,                  -- IMPROVEMENT_SURCHARGE
    234,                  -- DOLOCATIONID
    '2025-08-16 14:00:00',-- PICKUP_DATETIME
    1,                    -- VENDORID
    'N',                  -- STORE_AND_FWD_FLAG
    0.0,                  -- EXTRA
    25.0,                 -- FARE_AMOUNT
    33.8,                 -- TOTAL_AMOUNT
    1,                    -- PAYMENT_TYPE
    3.5,                  -- TRIP_DISTANCE
    'Downtown',           -- DROPOFF_ZONE
    1,                    -- RATECODEID
    123,                  -- PULOCATIONID
    'GREEN'               -- TAXI_COLOR
);


In [ ]:
CREATE DATABASE LAVRENIUK_TAXI_DEV CLONE LAVRENIUK_HW3;
